<a href="https://colab.research.google.com/github/jmg764/Molecular-Scalar-Coupling-Constant-Prediction-using-SchNet/blob/main/Molecular_Scalar_Coupling_Constant_Prediction_using_SchNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Molecular Scalar Coupling Constant Prediction using SchNet

In [ ]:
!pip install chainer-chemistry==0.5.0

     |████████████████████████████████| 81kB 3.4MB/s 
  Created wheel for chainer-chemistry: filename=chainer_chemistry-0.5.0-cp36-none-any.whl size=132520 sha256=3885ff9a869bc118f04d6f24791463ef1eced74c733913f0bd4ba413c06bf019
  Stored in directory: /root/.cache/pip/wheels/2e/32/e0/15d059a9218ee5d6306e124aa82b41a63c5fc61885fca277ae
Successfully built chainer-chemistry


In [ ]:
import numpy as np
import pandas as pd
import chainer
import chainer_chemistry

/usr/local/lib/python3.6/dist-packages/chainer_chemistry/__init__.py:12: UserWarning: A module chainer_chemistry.datasets was not imported, probably because RDKit is not installed. To install RDKit, please follow instruction in https://github.com/pfnet-research/chainer-chemistry#installation.
  UserWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
structures = pd.read_csv('/content/drive/MyDrive/DL Final Project/champs-scalar-coupling/structures.csv')
train = pd.merge(pd.read_csv('/content/drive/MyDrive/DL Final Project/champs-scalar-coupling/train.csv'),
                     pd.read_csv('/content/drive/MyDrive/DL Final Project/champs-scalar-coupling/scalar_coupling_contributions.csv'))

In [ ]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,fc,sd,pso,dso
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,83.0224,0.254579,1.25862,0.272010
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,-11.0347,0.352978,2.85839,-3.433600
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,-11.0325,0.352944,2.85852,-3.433870
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,-11.0319,0.352934,2.85855,-3.433930
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,83.0222,0.254585,1.25861,0.272013


In [ ]:
structures.head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


For each molecule, ```structures.csv``` lists the cartesian coordinates for each indexed atom

In [ ]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,fc,sd,pso,dso
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,83.0224,0.254579,1.25862,0.272010
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,-11.0347,0.352978,2.85839,-3.433600
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,-11.0325,0.352944,2.85852,-3.433870
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,-11.0319,0.352934,2.85855,-3.433930
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,83.0222,0.254585,1.25861,0.272013


The ```train.csv``` file contains: 


1.   ```molecule_name```: an id for the molecule under consideration
2.   ```atom_index_0``` and ```atom_index_1```: indices of the atom pairs which create a particular coupling
3.  ```type```: indicates the type of coupling interaction 
4.  ```scalar_coupling_constant```: our target variable




In [ ]:
print(f'Size of structures.csv: {structures.shape[0]} rows and {structures.shape[1]} columns')
print(f'Size of train.csv: {train.shape[0]} rows and {train.shape[1]} columns')
print('')
print(f'Number of distinct molecules in train.csv: {train.molecule_name.nunique()}')
print(f'List of unique atoms: {structures.atom.unique()}')



Size of structures.csv: 2358875 rows and 6 columns
Size of train.csv: 4659076 rows and 10 columns

Number of distinct molecules in train.csv: 85012
List of unique atoms: ['C' 'H' 'N' 'O' 'F']


In [ ]:
# Create validation and test set from provided train set
import random 

train_molecule_counts = train['molecule_name'].value_counts()
train_molecules = list(train_molecule_counts.index)

random.shuffle(train_molecules)

num_train = int(len(train_molecules) * 0.8)
test_valid_split = int(num_train + (int(len(train_molecules) - num_train))/2)

test_molecules = sorted(train_molecules[test_valid_split:])
valid_molecules = sorted(train_molecules[num_train:test_valid_split])
train_molecules = sorted(train_molecules[:num_train])

In [ ]:
new_train = train.query('molecule_name in @train_molecules')
new_valid = train.query('molecule_name in @valid_molecules')
new_test = train.query('molecule_name in @test_molecules')

new_train.sort_values('molecule_name', inplace=True)
new_valid.sort_values('molecule_name', inplace=True)
new_test.sort_values('molecule_name', inplace=True)

train_gp = new_train.groupby('molecule_name')
valid_gp = new_valid.groupby('molecule_name')
test_gp = new_test.groupby('molecule_name')
structures_groups = structures.groupby('molecule_name')

list_atoms = list(set(structures['atom']))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
from scipy.spatial import distance

class Graph:

    def __init__(self, points_df, list_atoms):

        self.points = points_df[['x', 'y', 'z']].values

        self._dists = distance.cdist(self.points, self.points)

        self.adj = self._dists < 1.5
        self.num_nodes = len(points_df)

        self.atoms = points_df['atom']
        dict_atoms = {at: i for i, at in enumerate(list_atoms)}

        atom_index = [dict_atoms[atom] for atom in self.atoms]
        one_hot = np.identity(len(dict_atoms))[atom_index]

        bond = np.sum(self.adj, 1) - 1
        bonds = np.identity(len(dict_atoms))[bond - 1]

        self._array = np.concatenate([one_hot, bonds], axis=1).astype(np.float32)

    @property
    def input_array(self):
        return self._array

    @property
    def dists(self):
        return self._dists.astype(np.float32)

In [ ]:
def create_graph(atoms, molecule_names, dataset, coordinates):
  graphs = list()
  targets = list()
  print('preprocessing molecules ...')
  for molecule in molecule_names:
      graphs.append(Graph(coordinates.get_group(molecule), atoms))
      targets.append(dataset.get_group(molecule))

  return graphs, targets



In [ ]:
len(train_molecules), len(train_gp), len(valid_molecules), len(valid_gp), len(test_molecules), len(test_gp)

(68009, 68009, 8501, 8501, 8502, 8502)

In [ ]:
train_graphs, train_targets = create_graph(list_atoms, train_molecules, train_gp, structures_groups)
valid_graphs, valid_targets = create_graph(list_atoms, valid_molecules, valid_gp, structures_groups)
test_graphs, test_targets = create_graph(list_atoms, test_molecules, test_gp, structures_groups)



preprocessing molecules ...
preprocessing molecules ...
preprocessing molecules ...


### Convert into chainer's dataset

In [ ]:
from chainer.datasets.dict_dataset import DictDataset

train_dataset = DictDataset(graphs=train_graphs, targets=train_targets)
valid_dataset = DictDataset(graphs=valid_graphs, targets=valid_targets)
test_dataset = DictDataset(graphs=test_graphs, targets=test_targets)

## Build SchNet Model

In [ ]:
from chainer import reporter
from chainer import functions as F
from chainer import links as L
from chainer_chemistry.links import SchNetUpdate
from chainer_chemistry.links import GraphLinear, GraphBatchNormalization
from chainer_chemistry.links.update.schnet_update import CFConv

class SchNetUpdateBN(SchNetUpdate):

    def __init__(self, *args, num_rbf, radius_resolution, gamma, **kwargs):
        super(SchNetUpdateBN, self).__init__(*args, **kwargs)
        self.num_rbf = num_rbf
        self.radius_resolution = radius_resolution
        self.gamma = gamma
        with self.init_scope():
            self.cfconv2 = CFConv(num_rbf=self.num_rbf, radius_resolution=self.radius_resolution, gamma=self.gamma, hidden_dim=512)
            self.bn = GraphBatchNormalization(args[0])

    def __call__(self, h, adj, **kwargs):
        v = self.linear[0](h)
        v = self.cfconv2(v, adj)
        v = self.linear[1](v)
        v = F.softplus(v)
        v = self.linear[2](v)
        return h + self.bn(v)

class SchNet(chainer.Chain):

    def __init__(self, num_layer=3, num_rbf=300, radius_resolution=0.1, gamma=10.0):
        super(SchNet, self).__init__()

        self.num_layer = num_layer
        self.num_rbf = num_rbf
        self.radius_resolution = radius_resolution
        self.gamma = gamma

        with self.init_scope():
            self.gn = GraphLinear(512)
            for l in range(self.num_layer):
                self.add_link('sch{}'.format(l), SchNetUpdateBN(512, num_rbf=self.num_rbf, radius_resolution=self.radius_resolution, gamma=self.gamma))

            self.interaction1 = L.Linear(128)
            self.interaction2 = L.Linear(128)
            self.interaction3 = L.Linear(4)

    def __call__(self, input_array, dists, pairs_index, targets):

        out = self.predict(input_array, dists, pairs_index)
        loss = F.mean_absolute_error(out, targets)
        reporter.report({'loss': loss}, self)
        return loss

    def predict(self, input_array, dists, pairs_index, **kwargs):

        h = self.gn(input_array)

        for l in range(self.num_layer):
            h = self['sch{}'.format(l)](h, dists)

        h = F.concat((h, input_array), axis=2)

        concat = F.concat([
            h[pairs_index[:, 0], pairs_index[:, 1], :],
            h[pairs_index[:, 0], pairs_index[:, 2], :],
            F.expand_dims(dists[pairs_index[:, 0],
                                pairs_index[:, 1],
                                pairs_index[:, 2]], 1)
        ], axis=1)

        h1 = F.leaky_relu(self.interaction1(concat))
        h2 = F.leaky_relu(self.interaction2(h1))
        out = self.interaction3(h2)

        return out


## SchNet Training Preparation

### Define Sampler

In [ ]:
from chainer.iterators import OrderSampler

class SameSizeSampler(OrderSampler):

    def __init__(self, structures_groups, moles, batch_size,
                 random_state=None, use_remainder=False):

        self.structures_groups = structures_groups
        self.moles = moles
        self.batch_size = batch_size
        if random_state is None:
            random_state = np.random.random.__self__
        self._random = random_state
        self.use_remainder = use_remainder

    def __call__(self, current_order, current_position):

        batches = list()

        atom_counts = pd.DataFrame()
        atom_counts['mol_index'] = np.arange(len(self.moles))
        atom_counts['molecular_name'] = self.moles
        atom_counts['num_atom'] = [len(self.structures_groups.get_group(mol))
                                   for mol in self.moles]

        num_atom_counts = atom_counts['num_atom'].value_counts()

        for count, num_mol in num_atom_counts.to_dict().items():
            if self.use_remainder:
                num_batch_for_this = -(-num_mol // self.batch_size)
            else:
                num_batch_for_this = num_mol // self.batch_size

            target_mols = atom_counts.query('num_atom==@count')['mol_index'].values
            random.shuffle(target_mols)

            devider = np.arange(0, len(target_mols), self.batch_size)
            devider = np.append(devider, 99999)

            if self.use_remainder:
                target_mols = np.append(
                    target_mols,
                    np.repeat(target_mols[-1], -len(target_mols) % self.batch_size))

            for b in range(num_batch_for_this):
                batches.append(target_mols[devider[b]:devider[b + 1]])

        random.shuffle(batches)
        batches = np.concatenate(batches).astype(np.int32)

        return batches

### Define Updater

In [ ]:
from chainer import training
from chainer.dataset import to_device

def coupling_converter(batch, device):

    list_array = list()
    list_dists = list()
    list_targets = list()
    list_pairs_index = list()

    with_target = 'fc' in batch[0]['targets'].columns

    for i, d in enumerate(batch):
        list_array.append(d['graphs'].input_array)
        list_dists.append(d['graphs'].dists)
        if with_target:
            list_targets.append(
                d['targets'][['fc', 'sd', 'pso', 'dso']].values.astype(np.float32))

        sample_index = np.full((len(d['targets']), 1), i)
        atom_index = d['targets'][['atom_index_0', 'atom_index_1']].values

        list_pairs_index.append(np.concatenate([sample_index, atom_index], axis=1))

    input_array = to_device(device, np.stack(list_array))
    dists = to_device(device, np.stack(list_dists))
    pairs_index = np.concatenate(list_pairs_index)

    array = {'input_array': input_array, 'dists': dists, 'pairs_index': pairs_index}

    if with_target:
      array['targets'] = to_device(device, np.concatenate(list_targets))

    return array

### Define Evaluator

In [ ]:
from chainer.training.extensions import Evaluator
from chainer import cuda
import os.path
from os import path

class TypeWiseEvaluator(Evaluator):

    def __init__(self, iterator, target, converter, device, name, lr, bs,
                 is_validate=False, is_submit=False):

        super(TypeWiseEvaluator, self).__init__(
            iterator, target, converter=converter, device=device)

        self.is_validate = is_validate
        self.is_submit = is_submit
        self.name = name
        self.lr = lr
        self.bs = bs

    def calc_score(self, df_truth, pred):

        target_types = list(set(df_truth['type']))

        diff = df_truth['scalar_coupling_constant'] - pred

        scores = 0
        metrics = {}

        for target_type in target_types:

            target_pair = df_truth['type'] == target_type
            score_exp = np.mean(np.abs(diff[target_pair]))
            scores += np.log(score_exp)

            metrics[target_type] = scores

        metrics['ALL_LogMAE'] = scores / len(target_types)

        observation = {}
        with reporter.report_scope(observation):
            reporter.report(metrics, self._targets['main'])

        return observation

    def evaluate(self):
        iterator = self._iterators['main']
        eval_func = self._targets['main']

        iterator.reset()
        it = iterator

        y_total = []
        t_total = []

        for batch in it:
            in_arrays = self.converter(batch, self.device)
            with chainer.no_backprop_mode(), chainer.using_config('train', False):
                y = eval_func.predict(**in_arrays)

            y_data = cuda.to_cpu(y.data)
            y_total.append(y_data)
            t_total.extend([d['targets'] for d in batch])

        df_truth = pd.concat(t_total, axis=0)
        y_pred = np.sum(np.concatenate(y_total), axis=1)

        if self.is_submit:
          results = self.calc_score(df_truth, y_pred)
          if path.exists('test_results.csv'):
            test_results = pd.read_csv('test_results.csv')
            test_results.loc[len(test_results)] = [lr, results['test/main/ALL_LogMAE']]
            test_results.to_csv('test_results.csv', index=False)
          else:
            test_results = pd.DataFrame()
            test_results['learning rate'] = [lr]
            # test_results['batch size'] = [bs]
            test_results['All_LogMAE'] = [results['test/main/ALL_LogMAE']]
            test_results.to_csv('test_results.csv', index=False)

          submit = pd.DataFrame()
          submit['id'] = df_truth['id']
          submit['predicted_scc'] = y_pred
          submit['true_scc'] = df_truth['scalar_coupling_constant']
          submit.drop_duplicates(subset='id', inplace=True)
          submit.sort_values('id', inplace=True)
          submit.to_csv('kernel_schnet.csv', index=False)

        if self.is_validate:
          return self.calc_score(df_truth, y_pred)
            
            


        return {}


### Other Extensions

In [ ]:
def other_extensions(trainer):
  # Learning rate scheduler 
  trainer.extend(training.extensions.ExponentialShift('alpha', 0.99999))

  # Extension which turns off normalization after the second epoch
  from chainer.training import make_extension

  def stop_train_mode(trigger):
      @make_extension(trigger=trigger)
      def _stop_train_mode(_):
          chainer.config.train = False
      return _stop_train_mode

  trainer.extend(stop_train_mode(trigger=(1, 'epoch')))

  trainer.extend(
      training.extensions.observe_value(
          'alpha', lambda tr: tr.updater.get_optimizer('main').alpha))


  # Report metrics
  trainer.extend(training.extensions.LogReport())
  # trainer.extend(training.extensions.ProgressBar())
  trainer.extend(training.extensions.PrintReport(
      ['epoch', 'elapsed_time', 'main/loss', 'valid/main/ALL_LogMAE', 'alpha']))


## Hyperparameter Tuning

In [ ]:
from chainer import optimizers

def run_model(learning_rate, num_layer=3, num_rbf=300, radius_resolution=0.1, gamma=10.0):
  model = SchNet(num_layer, num_rbf, radius_resolution, gamma)
  model.to_gpu(device=0)
  
  batch_size = 4
  train_sampler = SameSizeSampler(structures_groups, train_molecules, batch_size)
  valid_sampler = SameSizeSampler(structures_groups, valid_molecules, batch_size,
                                  use_remainder=True)
  test_sampler = SameSizeSampler(structures_groups, test_molecules, batch_size,
                                use_remainder=True)
  
  train_iter = chainer.iterators.SerialIterator(
    train_dataset, batch_size, order_sampler=train_sampler)

  valid_iter = chainer.iterators.SerialIterator(
      valid_dataset, batch_size, repeat=False, order_sampler=valid_sampler)

  test_iter = chainer.iterators.SerialIterator(
      test_dataset, batch_size, repeat=False, order_sampler=test_sampler)

  optimizer = optimizers.Adam(alpha=learning_rate)
  optimizer.setup(model)
  updater = training.StandardUpdater(train_iter, optimizer,
                                    converter=coupling_converter, device=0)
  trainer = training.Trainer(updater, (25, 'epoch'), out="result")

  trainer.extend(
      TypeWiseEvaluator(iterator=valid_iter, target=model, converter=coupling_converter, 
                        name='valid', device=0, lr=learning_rate, bs=batch_size, is_validate=True))
  trainer.extend(
      TypeWiseEvaluator(iterator=test_iter, target=model, converter=coupling_converter,
                        name='test', device=0, lr=learning_rate, bs=batch_size, is_submit=True))

  other_extensions(trainer)

  chainer.config.train = True
  trainer.run()

### Varied learning rate 

In [ ]:
learning_rates = [1e-3, 5e-3, 1e-2]

for lr in learning_rates:
  run_model(lr)

### Varied radius resolution

In [ ]:
radius_resolutions = [0.05, 0.075, 0.125, 0.15]

for rr in radius_resolutions:
  run_model(lr=5e-3, radius_resolution=rr)